In [96]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.tools.visualization import plot_histogram

import random


In [97]:
ndim = 2

In [98]:
from qiskit.quantum_info import Pauli
from itertools import product

# create n-qubit pauli operator set using cartesian product
def create_pauli(n):
    pauli_set = []
    for i in product(['I', 'X', 'Y', 'Z'], repeat=n):
        pauli_set.append(Pauli(''.join(i)))
    return pauli_set

# create 2-qubit pauli operator set
pauli_set_2n = create_pauli(ndim)
assert not(ndim ==2) or len(pauli_set_2n) == 16

In [99]:
import numpy as np
# create n-qubit stabilizer states 
# by definition, the stabilizer states are created by Clifford gates acting on |0\>^{otimes n}
# |0>, |1>, |+>, |->, |i>, |-i>

# XXX I cannot figure outa way to generate the stabilizer states directly
# thus, first we need to generate the Clifford set

# create n-qubit Clifford set
# the Clifford set is generated by H, S, CNOT

# its kind of silly, but we could use random Cliffords
from qiskit.quantum_info import random_clifford
from qiskit.quantum_info import Statevector

stabilizer_states = []
N = 2**5*ndim # defines the number of random Clifford circuits to generate, larger is required for larger ndim
for i in range(N):
    # run random clifford 
    qc = QuantumCircuit(ndim)
    qc.append(random_clifford(ndim), range(ndim))
    # circuit to state
    state = Statevector.from_instruction(qc)
    # append to set
    stabilizer_states.append(state)
    # print(state)

# drop duplicates (up to a global phase)
# use Statevector.equiv to check if two states are equivalent
for i in range(len(stabilizer_states)):
    if stabilizer_states[i] is None:
        continue
    for j in range(i+1, len(stabilizer_states)):
        if stabilizer_states[j] is None:
            continue
        if stabilizer_states[i].equiv(stabilizer_states[j]):
            stabilizer_states[j] = None
stabilizer_states = [x for x in stabilizer_states if x is not None]
print(len(stabilizer_states))

40


AttributeError: type object 'Statevector' has no attribute 'to_density_matrix'

In [ ]:
# https://arxiv.org/abs/1609.07488

# next need to create A and B matrices
# initialize A with rows = len(pauli_operators), cols = len(stabilizer_states)
A = np.zeros((len(pauli_set_2n), len(stabilizer_states)))
# A_ji =Tr(P_j \sigma_i), where P_j is the j-th Pauli operator, \sigma_i is the i-th stabilizer state
for i in range(len(stabilizer_states)):
    for j in range(len(pauli_set_2n)):
        # A[j,i] = np.trace(pauli_set_2n[j].to_matrix() @ stabilizer_states[i].data)
        # A[j,i] = stabilizer_states[i].evolve(pauli_set_2n[j]).trace() #IDK why this is not working
        # use Statevector.expectation_value
        A[j,i] = stabilizer_states[i].expectation_value(pauli_set_2n[j])
print(A)


[[ 1.  1.  1.  1.  1.  1.]
 [-1.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  1. -1.]
 [ 0.  1. -1.  0.  0.  0.]]


In [9]:
N = 4 #40
circ = QuantumCircuit(N, N)

# Initialize with a Hadamard layer
circ.h(range(N))
# Apply some random CNOT and T gates
qubit_indices = [i for i in range(N)]
for i in range(10):
    control, target, t = random.sample(qubit_indices, 3)
    circ.cx(control, target)
    circ.t(t)
circ.measure(range(N), range(N))

In [12]:
circ.draw()

┌───┐     ┌───┐          ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐     ┌─┐      
q_0: ┤ H ├──■──┤ T ├──────────┤ X ├┤ X ├┤ T ├┤ X ├┤ T ├┤ X ├─────┤M├──────
     ├───┤  │  └───┘┌───┐     └─┬─┘└─┬─┘└───┘└─┬─┘├───┤└─┬─┘     └╥┘┌─┐   
q_1: ┤ H ├──┼────■──┤ T ├──■────┼────■────■────┼──┤ X ├──■────────╫─┤M├───
     ├───┤┌─┴─┐┌─┴─┐├───┤  │    │       ┌─┴─┐  │  └─┬─┘ ┌─┐       ║ └╥┘   
q_2: ┤ H ├┤ X ├┤ X ├┤ X ├──┼────■───────┤ X ├──■────┼───┤M├───────╫──╫────
     ├───┤├───┤├───┤└─┬─┘┌─┴─┐┌───┐┌───┐├───┤       │   └╥┘ ┌───┐ ║  ║ ┌─┐
q_3: ┤ H ├┤ T ├┤ T ├──■──┤ X ├┤ T ├┤ T ├┤ T ├───────■────╫──┤ T ├─╫──╫─┤M├
     └───┘└───┘└───┘     └───┘└───┘└───┘└───┘            ║  └───┘ ║  ║ └╥┘
c: 4/════════════════════════════════════════════════════╩════════╩══╩══╩═
                                                         2        0  1  3

In [10]:
# Create statevector method simulator
statevector_simulator = AerSimulator(method='statevector')

# Transpile circuit for backend
tcirc = transpile(circ, statevector_simulator)

# Try and run circuit
statevector_result =  statevector_simulator.run(tcirc, shots=1).result()
print('This succeeded?: {}'.format(statevector_result.success))
print('Why not? {}'.format(statevector_result.status))

This succeeded?: True
Why not? COMPLETED


In [11]:
# Create extended stabilizer method simulator
extended_stabilizer_simulator = AerSimulator(method='extended_stabilizer')

# Transpile circuit for backend
tcirc = transpile(circ, extended_stabilizer_simulator)

extended_stabilizer_result = extended_stabilizer_simulator.run(tcirc, shots=1).result()
print('This succeeded?: {}'.format(extended_stabilizer_result.success))

This succeeded?: True
